In [348]:
from misc_functions            import *
from db.Database import *
# import db.Area as Area
from db.Area import *
from db.AreaExit import *

from db.MudMap import MudMap

import networkx, json, os, re

In [349]:
area_select_statement = 'v_areas_for_graph where name like "%Something disin%"'
areas = Area.raw('select * from ' + area_select_statement)

In [350]:
area = areas[0]
print(area.id, area.name, area.description)

3049 Something disintegrates . . . . Reverberent Cavern  Each step, breath and movement echoes with purity back to you. This is another large cavern, similar to the chiming cavern below, but with glassy smooth walls. The effect is to cause every sound to become magnified tenfold. In the middle of this cave though is a very disturbing sight. Many, many bones are piled here, cattle, sheep, humans, gnolls. In fact it appears anything that can be eaten. Looking around you see how the cavern is in fact a nest, an immense one, but a nest nonetheless.


In [351]:
def mock_area_string(area):
    area_str = area.name + '\n\r\n\r' + area.description.replace("  ","\n\r\n\r").replace(". ", ".\n") + "\n\rObvious exits: out, west, south.\n\rYou see Derp.\n\rYou see a war horse, a deaconess.\n\rYou see 10 gold coins.\n\r"
    return f"{area_str}"

In [352]:
print(mock_area_string(area))

Something disintegrates . . . .

Reverberent Cavern

Each step, breath and movement echoes with purity back to you.
This is another large cavern, similar to the chiming cavern below, but with glassy smooth walls.
The effect is to cause every sound to become magnified tenfold.
In the middle of this cave though is a very disturbing sight.
Many, many bones are piled here, cattle, sheep, humans, gnolls.
In fact it appears anything that can be eaten.
Looking around you see how the cavern is in fact a nest, an immense one, but a nest nonetheless.
Obvious exits: out, west, south.
You see Derp.
You see a war horse, a deaconess.
You see 10 gold coins.



In [353]:
area_match_regex = "(?si)(.+?)(Obvious exits: .+?\.)\n?\r?(You see .+?\.)?\n?\r?(You see .+?\.)?\n?\r?(You see .+?\.)?\n?\r?(You see .+?\.)?\n?\r?(You see .+?\.)?\n?\r?"

In [354]:
M_obj = re.search(area_match_regex, mock_area_string(area), re.DOTALL)
print(M_obj)

print(M_obj.group(1))

<re.Match object; span=(0, 660), match='Something disintegrates . . . .\n\r\n\rReverberen>
Something disintegrates . . . .

Reverberent Cavern

Each step, breath and movement echoes with purity back to you.
This is another large cavern, similar to the chiming cavern below, but with glassy smooth walls.
The effect is to cause every sound to become magnified tenfold.
In the middle of this cave though is a very disturbing sight.
Many, many bones are piled here, cattle, sheep, humans, gnolls.
In fact it appears anything that can be eaten.
Looking around you see how the cavern is in fact a nest, an immense one, but a nest nonetheless.



In [355]:
groups = M_obj.groups()

area_content_raw = groups[0].strip()
area_description = re.search("(?si)^.*\n\r?\n\r?(.*)$", area_content_raw.strip()).groups()[0]

area_title_raw = area_content_raw.replace(area_description, "").strip()

# get the last line of the title
area_title = re.search("(?si)^(?:.+M] )?(.*)$", area_title_raw.split("\n")[-1]).groups()[0]
area_title

'In fact it appears anything that can be eaten.'

In [356]:
def convert_group_to_string(M_obj):
    groups = M_obj.groups()

    area_content_raw = groups[0].strip()

    area_description = re.search("(?si)^.*\n\r?(.*)$", area_content_raw.strip()).groups()[0]
    area_title_raw = area_content_raw.replace(area_description, "").strip()

    # get the last line of the first group to get the area title
    area_title = re.search("(?si)^(?:.+M] )?(.*)$", area_title_raw.split("\n")[-1]).groups()[0]
    
    area_exits = groups[1]
    area_signs = groups[2]
    area_characters = groups[3]
    area_mobs = groups[4]
    area_items = groups[5]

    return f"{area_title}\n\r\n\r{area_description}\n\r{area_exits}\n\r{area_signs}\n\r{area_characters}\n\r{area_mobs}\n\r"


In [357]:
print(convert_group_to_string(M_obj))

In fact it appears anything that can be eaten.

Looking around you see how the cavern is in fact a nest, an immense one, but a nest nonetheless.
Obvious exits: out, west, south.
You see Derp.
You see a war horse, a deaconess.
You see 10 gold coins.



In [358]:
def convert_area_to_string(area):
    area_str = mock_area_string(area)
    print(area_str)
    M_obj = re.search(area_match_regex, area_str, re.DOTALL)
    if M_obj:
        print(convert_group_to_string(M_obj))
    else:
        print("No match for area: ", area.name)

In [359]:
# convert_area_to_string(areas[0])
# convert_area_to_string(areas[1])
# convert_area_to_string(areas[2])
# convert_area_to_string(areas[3])
convert_area_to_string(areas[0])

Something disintegrates . . . .

Reverberent Cavern

Each step, breath and movement echoes with purity back to you.
This is another large cavern, similar to the chiming cavern below, but with glassy smooth walls.
The effect is to cause every sound to become magnified tenfold.
In the middle of this cave though is a very disturbing sight.
Many, many bones are piled here, cattle, sheep, humans, gnolls.
In fact it appears anything that can be eaten.
Looking around you see how the cavern is in fact a nest, an immense one, but a nest nonetheless.
Obvious exits: out, west, south.
You see Derp.
You see a war horse, a deaconess.
You see 10 gold coins.

In fact it appears anything that can be eaten.

Looking around you see how the cavern is in fact a nest, an immense one, but a nest nonetheless.
Obvious exits: out, west, south.
You see Derp.
You see a war horse, a deaconess.
You see 10 gold coins.

